In [57]:
import gzip
from collections import defaultdict
import random 
from sklearn import linear_model
import string
import json

In [6]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')

Would-read baseline: just rank which books are popular and which are not, and return '1' if a book is among the top-ranked


In [7]:
bookCount = defaultdict(int)
totalRead = 0
userBook = defaultdict(set)
bookUser = defaultdict(set)
num = 0
train = []
vali = []

for user,book,rat in readCSV("train_Interactions.csv.gz"):
    if num < 190001:
        bookCount[book] += 1
        totalRead += 1
        userBook[user].add(book)
        bookUser[book].add(user)
        train.append((user, book, rat))
    else: vali.append((user, book, rat))
    num += 1
        
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

In [8]:
print(len(bookCount))
print(len(mostPopular))
print(mostPopular[:10])

7169
7169
[(381, 'b25543219'), (319, 'b76915592'), (313, 'b21517939'), (273, 'b75885962'), (271, 'b55315814'), (267, 'b52453648'), (263, 'b02830492'), (249, 'b39244888'), (248, 'b25118404'), (235, 'b75743714')]


In [9]:
size = len(vali)
for i in range(size):
    user = vali[i][0]
    count = 0
    while count < len(bookCount):
        ind = random.randint(0,len(mostPopular)-1)
        if mostPopular[ind][1] not in userBook[user]:
            temp = (user, mostPopular[ind][1], -1)
            vali.append(temp)
            break;
        count += 1

In [10]:
def getAccuracy(thre):
    topBooks = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        topBooks.add(i)
        if count > totalRead/thre: break
    count = 0
    for user, book, rating in vali:
        if book in topBooks and int(rating) >= 0: count += 1
        if book not in topBooks and int(rating) < 0: count += 1
    return (count/len(vali))

Problem1: when the threshold is 2, the accuracy is 0.64

In [11]:
print(getAccuracy(2))

0.6482148214821483


Problem2: when the threshold is 1,79, the accuracy is 0.65, which is slightly higher than using 2 as the threshold

In [12]:
print(getAccuracy(1.75))

0.6526652665266527


Problem3: Jaccard similarity

In [13]:
def Jaccard(s1, s2):
    c1 = len(s1.intersection(s2))
    c2 = len(s1.union(s2))
    
    return (c1/c2)

In [14]:
simi = []
for i in range(len(vali)):
    user = vali[i][0]
    book = vali[i][1]
    rating = vali[i][2]
    high = 0
    for b in userBook[user]:
        high = max(high, Jaccard(bookUser[b], bookUser[book]))
    simi.append(high)


In [15]:
def getAccuracy(thre):
    count = 0
    #print(thre)
    for i in range(len(vali)):
        user = vali[i][0]
        book = vali[i][1]
        rating = vali[i][2]
        if simi[i] > thre and int(rating) >= 0: count += 1
        if simi[i] < thre and int(rating) < 0: count += 1
    return (count/len(vali))

In [23]:
# for i in range(1,30):
#     print(getAccuracy(i*0.001))

Problem 4:

In [16]:
# Combine two models
def getCombinedAccuracy(topBookThre, jaccardThre):
    topBooks = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        topBooks.add(i)
        if count > totalRead/topBookThre: break
            
    count = 0
    for i in range(len(vali)):
        user, book, rating = vali[i][0], vali[i][1], vali[i][2]
        if simi[i] > jaccardThre and int(rating) >= 0:
            count += 1
        elif int(rating) < 0:
            count += 1
            
    return (count / len(vali))

def predict(topBooks, userBook, bookUser, user, book, jaccardThre=0.01):
    high = 0
    for b in userBook[user]:
        high = max(high, Jaccard(bookUser[b], bookUser[book]))
        
    if book in topBooks and high > jaccardThre:
        return 1
    else:
        return 0

In [17]:
getCombinedAccuracy(1.0, 0.002)

0.9601960196019602

In [18]:
print(totalRead)

190001


Problem5: (my Kaggle username is Manxue Li)

In [19]:
predictions = open("predictions_Read.txt", 'w')

topBooks = set()
topBookThre=1.75
count = 0
for ic, i in mostPopular:
    count += ic
    topBooks.add(i)
    if count > totalRead/topBookThre: break
        
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    if predict(topBooks, userBook, bookUser, u, b) == 1 :
        predictions.write(u + '-' + b + ",1\n")
    else:
        predictions.write(u + '-' + b + ",0\n")
predictions.close()

 Category prediction baseline: Just consider some of the most common words from each category


In [20]:
catDict = {
  "children": 0,
  "comics_graphic": 1,
  "fantasy_paranormal": 2,
  "mystery_thriller_crime": 3,
  "young_adult": 4
}

In [21]:
predictions = open("predictions_Category.txt", 'w')
predictions.write("userID-reviewID,prediction\n")
for l in readGz("test_Category.json.gz"):
  cat = catDict['fantasy_paranormal'] # If there's no evidence, just choose the most common category in the dataset
  words = l['review_text'].lower()
  if 'children' in words:
    cat = catDict['children']
  if 'comic' in words:
    cat = catDict['comics_graphic']
  if 'fantasy' in words:
    cat = catDict['fantasy_paranormal']
  if 'mystery' in words:
    cat = catDict['mystery_thriller_crime']
  if 'young' in words:
    cat = catDict['young_adult']
  predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(cat) + "\n")

predictions.close()

In [28]:
# get feature vector
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for l in readGz("test_Category.json.gz"):
    r = ''.join([c for c in l['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

Problem6:

In [64]:
### Just take the most popular words...
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
print(counts[:10])

[(72087, 'the'), (43996, 'and'), (38613, 'a'), (36966, 'to'), (36113, 'i'), (31713, 'of'), (21711, 'is'), (20493, 'in'), (19948, 'it'), (19102, 'this')]


In [79]:
def feature(datum, words, wordId):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

Problem7: feature vector and label vector

In [66]:
train_X = []
train_y = []
test_X = []
test_y = []
num = 0

for l in readGz("train_Category.json.gz"):
    if num < 190000:
        train_X.append(feature(l))
        train_y.append(l['genreID'])
    else:
        test_X.append(feature(l))
        test_y.append(l['genreID'])
    num += 1

In [83]:
def predictCat(counts, dictSize, c):
    # get the top popular words
    words = [x[1] for x in counts[:dictSize]]
    wordId = {}
    wordSet = set(words)
    num = 0
    for w in words:
        wordId[w] = num
        num += 1
    
    train_X = []
    train_y = []
    test_X = []
    test_y = []
    num = 0

    for l in readGz("train_Category.json.gz"):
        if num < 190000:
            train_X.append(feature(l, words, wordId))
            train_y.append(l['genreID'])
        else:
            test_X.append(feature(l, words, wordId))
            test_y.append(l['genreID'])
        num += 1
        
    # train the model and generate prediction  
    mod = linear_model.LogisticRegression(C=c)
    mod.fit(train_X, train_y)
    pred = mod.predict(test_X)
    return pred
    
# calculate accuracy
def getAccuracy(pred):
    count = 0
    for i in range(len(test_X)):
        if pred[i] == test_y[i]: count += 1
    return (count/len(test_y))

In [78]:
pred = predictCat(counts, 1000, 1)

/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Problem7: this is the accuracy of the model, 0.6417

In [95]:
print(getAccuracy(pred))

0.6417


In [85]:
pred2 = predictCat(counts, 2000, 10)
pred3 = predictCat(counts, 2000, 0.1)

/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [89]:
pred4 = predictCat(counts, 1000, 10)
pred5 = predictCat(counts, 1000, 0.1)

/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [91]:
pred6 = predictCat(counts, 1000, 100)
pred7 = predictCat(counts, 1000, 0.01)

/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureW

In [93]:
pred8 = predictCat(counts, 1500, 10)
pred9 = predictCat(counts, 1500, 0.1)

/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/clairlmx/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [94]:
# 1000, 1: 0.6417
print(getAccuracy(pred))
print(getAccuracy(pred2))
print(getAccuracy(pred3))
print(getAccuracy(pred4))
print(getAccuracy(pred5))
print(getAccuracy(pred6))
print(getAccuracy(pred7))
print(getAccuracy(pred8))
print(getAccuracy(pred9))

0.6417
0.6416
0.6407
0.6416
0.6407
0.6402
0.6376
0.6416
0.6407
